In [3]:
#titanic_kfp_pipeline.ipynb
#Copyright 2020 Google LLC. 
#This software is provided as-is, without warranty or representation for any use or purpose. 
#Your use of it is subject to your agreements with Google.
#Author: whjang@google.com

PROJECT_ID = “aiplatformdemo”
IMAGE_PREFIX = “whjang-titanic”
PREPROC_DIR = “preprocess”
TRAIN_DIR = “train”

WORK_BUCKET = “gs://ispark-example-bucket”
RAW_CSV_GCS_URI = WORK_BUCKET + “/rawdata/datasets_11657_16098_train.csv”
PREPROC_CSV_GCS_URI = WORK_BUCKET + “/preprocdata/processed_train.csv”
ACC_CSV_GCS_URI = WORK_BUCKET + “/latestacc/accuracy.csv”
MODEL_PKL_GCS_URI = WORK_BUCKET + “/model/model.pkl”
STAGE_GCS_FOLDER = WORK_BUCKET + “/stage”

AIPJOB_TRAINER_GCS_PATH = WORK_BUCKET + “/train/titanic_train.tar.gz”
AIPJOB_OUTPUT_GCS_PATH = WORK_BUCKET + “/train/output/”

import os
os.environ[‘PROJECT_ID’] = PROJECT_ID
os.environ[‘IMAGE_PREFIX’] = IMAGE_PREFIX
os.environ[‘PREPROC_DIR’] = PREPROC_DIR
os.environ[‘TRAIN_DIR’] = TRAIN_DIR
os.environ[‘WORK_BUCKET’] = WORK_BUCKET
os.environ[‘RAW_CSV_GCS_URI’] = RAW_CSV_GCS_URI
os.environ[‘PREPROC_CSV_GCS_URI’] = PREPROC_CSV_GCS_URI
os.environ[‘ACC_CSV_GCS_URI’] = ACC_CSV_GCS_URI
os.environ[‘MODEL_PKL_GCS_URI’] = MODEL_PKL_GCS_URI
os.environ[‘STAGE_GCS_FOLDER’] = STAGE_GCS_FOLDER
os.environ[‘AIPJOB_TRAINER_GCS_PATH’] = AIPJOB_TRAINER_GCS_PATH
os.environ[‘AIPJOB_OUTPUT_GCS_PATH’] = AIPJOB_OUTPUT_GCS_PATH

!echo $PROJECT_ID
!echo $IMAGE_PREFIX
!echo $PREPROC_DIR
!echo $TRAIN_DIR
!echo $WORK_BUCKET
!echo $RAW_CSV_GCS_URI
!echo $PREPROC_CSV_GCS_URI
!echo $ACC_CSV_GCS_URI
!echo $MODEL_PKL_GCS_URI
!echo $STAGE_GCS_FOLDER
!echo $AIPJOB_TRAINER_GCS_PATH
!echo $AIPJOB_OUTPUT_GCS_PATH

SyntaxError: invalid character in identifier (<ipython-input-3-55bc039f8a5d>, line 7)